In [1]:
# Import Libraries 
import os 
import copy,glob
import pandas as pd 
import numpy as np 
import warnings
import tensorflow as tf 
import keras 


import matplotlib.pyplot as plt 
import seaborn as sns

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
import cv2

from tensorflow.keras.models import Model, load_model,Sequential
from keras.layers import Flatten,Dense
from keras.layers import Dense,GlobalAveragePooling2D,Dropout,BatchNormalization,MaxPooling2D,GlobalMaxPooling2D
from keras.utils import np_utils
from keras .callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras import regularizers
from tensorflow.keras.optimizers import Adam,Adamax

from tensorflow.keras.applications import ResNet50V2, Xception
from keras.applications.mobilenet_v2 import MobileNetV2

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, f1_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.utils import shuffle

# Loading Images

In [2]:
train_gen=ImageDataGenerator(rescale=1/255)
valid_gen=ImageDataGenerator(rescale=1/255,validation_split=0.4)

train_data=train_gen.flow_from_directory("C:/Users/ismai/OneDrive/Documents/GitHub/Monkey_pox_detection/Data/Fold1/Fold1/Fold1/Train",target_size=(224,224),
                                         shuffle=True,class_mode='binary',batch_size=64)
valid_data=valid_gen.flow_from_directory("C:/Users/ismai/OneDrive/Documents/GitHub/Monkey_pox_detection/Data/Fold1/Fold1/Fold1/Val",target_size=(224,224),
                                         shuffle=True,class_mode='binary')

Found 2142 images belonging to 2 classes.
Found 420 images belonging to 2 classes.


In [3]:
# call Back 
import keras.callbacks


checkpoint_filepath='{epoch:2d}.h5'
chechpoint_dir=os.path.dirname(checkpoint_filepath)

model_chcekpoint_callback=keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True 
)

earlystop=EarlyStopping(monitor='val_accuracy',
                        min_delta=0,
                        patience=15,
                        verbose=1,
                        mode='auto')

reduce_lr=ReduceLROnPlateau(monitor='val_accuracy',
                            factor=0.2,
                            patience=4,
                            verbose=1,
                            min_delta=0.0001)

cb=[earlystop,reduce_lr,model_chcekpoint_callback]

# ResNet50v2

In [4]:
base_model_resnet=ResNet50V2(include_top=False,
                             input_shape=(224,224,3))
base_model_resnet.trainable=False

model_resnet=Sequential([base_model_resnet,GlobalAveragePooling2D(),
                         Dense(256,activation='relu'),
                         BatchNormalization(),
                         Dense(164,activation='relu'),
                         BatchNormalization(),
                         Dense(1,activation='sigmoid')])

model_resnet.summary()

InternalError: cudaGetDevice() failed. Status: initialization error